In [1]:
import pandas as pd
import os
import plotly.graph_objects as go

In [2]:
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

import seaborn as sns
sns.set_context('poster')
sns.set(rc={'figure.figsize': (16., 9.)})
sns.set_style('whitegrid')
import plotly.express as px
import plotly.graph_objects as go

In [3]:
os.listdir('../data')

['.~lock.diagnos.csv#',
 'ambientales',
 'at_primaria',
 'cal_lunar.csv',
 'codigos_com',
 'codigos_com.csv',
 'codigos_provincia.csv',
 'cod_pro_com.csv',
 'diagnos.csv',
 'eurostats_descartado',
 'icd10.csv',
 'ind_pobreza.csv',
 'luzmesespr.csv',
 'morbilidad',
 'morbilidad_Fgen.csv',
 'pobactiva.csv',
 'poblacion_prov.csv',
 'pob_activa_acum.csv',
 'pob_comunidad.csv',
 'pob_sex_pro_media.csv',
 'psico_atprimaria.csv',
 'renta.csv',
 'socioecon',
 'suicidio',
 'tasas_empleo.csv']

# Luna lunera

In [4]:
morbi = pd.read_csv('../data/morbilidad_Fgen.csv')

In [5]:
morbi.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,Edad,Dias,Factor,code,...,fecha_al,fecha_ing,year,month,day,Entrada,Alta,SexNom,ProvHosp_nom,ProvResi_nom
201380,8,2,8,1,F4323,1,51,13,1.02752,F43,...,2017-08-03,2017-07-21,2017,7,21,Ordinario,Curación,Mujer,Barcelona,Barcelona
203014,8,2,8,2,F4325,1,16,1,1.13239,F43,...,2017-11-10,2017-11-09,2017,11,9,Urgente,Curación,Mujer,Barcelona,Barcelona
404007,9,2,9,2,F323,1,76,1,1.00000,F32,...,2019-06-28,2019-06-27,2019,6,27,Urgente,Curación,Mujer,Burgos,Burgos


In [6]:
morbi = morbi[morbi['fecha_ing']>='2016']

In [7]:
dias = morbi.groupby(['fecha_ing']).agg({'Sexo':'count'}).reset_index().rename({'Sexo':'total'},axis=1)

In [8]:
dias.sample(3)

,fecha_ing,total
978,2018-09-05,358
853,2018-05-03,362
1150,2019-02-24,185


In [9]:
dias.info('cal_lunar.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fecha_ing  1460 non-null   object
 1   total      1460 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 22.9+ KB


In [10]:
lunas = pd.read_csv('../data/cal_lunar.csv')

In [11]:
lunas.sample()

,fecha,moon,fecha_Date
116,07/05/2018,Cuarto Menguante,2018-05-07


In [12]:
lunas_fecha = {}
for i,r in lunas.iterrows():
    
    lunas_fecha[r.fecha_Date] = r.moon

In [13]:
dias['luna'] = dias.fecha_ing.map(lunas_fecha)

In [14]:
dias.sample(5)

,fecha_ing,total,luna
669,2017-10-31,380,NaN
836,2018-04-16,400,NaN
817,2018-03-28,338,NaN
652,2017-10-14,169,NaN
72,2016-03-13,188,NaN


In [15]:
#dias.fillna(method='ffill',inplace = True)

In [16]:
ing_lunas = dias.groupby(['luna']).agg({'total':'sum','fecha_ing':'min'}).sort_values('fecha_ing').reset_index().drop('fecha_ing',axis=1)

In [17]:
ing_lunas

,luna,total
0,Cuarto Menguante,14579
1,Luna Nueva,14970
2,Cuarto Creciente,14341
3,Luna Llena,14604


In [18]:
#ing_lunas.to_csv('../output/total_lunas.csv',index=False)

In [48]:
fig_luna = px.histogram(ing_lunas, x="luna",y='total',color = 'luna')
fig_luna.show()

In [49]:
fig_luna.write_html('../output/luna.html')
    

In [21]:
edad = {'00-14':0, '15-34':15, '35-64':35, '65 +':65}

In [22]:
morbil = pd.read_csv('../data/morbilidad_Fgen.csv')

In [23]:
morbil.fecha_al.str[5:7].value_counts()

05    39526
03    38867
07    38187
06    38156
10    37371
04    37064
02    36074
08    35783
09    34884
01    34763
11    34734
12    33520
Name: fecha_al, dtype: int64

In [24]:
diag = pd.read_csv('../data/diagnos.csv')

In [25]:
diag_dic = {}
for i,r in diag.iterrows():
    diag_dic[r.cap] = r.title
morbil['title'] = morbil.cap.map(diag_dic)

# sal sol

In [26]:
luz = pd.read_csv('../data/luzmesespr.csv')

In [27]:
luz.sample(3)

,mes,Provincia,horas,cod,mes_n
115,Marzo,Castellón,11.95,12,3
50,Enero,Ceuta,10.04,51,1
491,Octubre,León,11.06,24,10


In [28]:
luz_tot = luz.groupby(['mes','mes_n']).agg({'horas':'mean'}).round(2).reset_index().sort_values('mes_n')

In [29]:
luz_tot.sample()

,mes,mes_n,horas
6,Junio,6,14.97


In [30]:
#dropeamos 2019 de morbilidad porque no están los datos completos de diciembre y queremos sacar una media
morbi_ = morbi[morbi['year']!=2019]

In [31]:
morbi_.year.value_counts()

2017    111907
2016    108935
2018    107384
Name: year, dtype: int64

In [32]:
mes_year = morbi.groupby(['year','month']).agg({'day':'count'}).reset_index().rename({'month':'mes_n','day':'count'},axis=1)

In [33]:
mes_year.sample()

,year,mes_n,count
7,2016,8,8728


In [34]:
mes_luz = mes_year.merge(luz_tot,on = 'mes_n')

In [35]:
ingresos_mes = morbi_.groupby('month').agg({'day':'count'}).reset_index().rename({'month':'mes_n','day':'count'},axis=1)

In [36]:
ingresos_mes_luz = ingresos_mes.merge(luz_tot,on='mes_n')

In [37]:
#ajustamos escalas para ver gráfico
ingresos_mes_luz['ing_mil'] = (ingresos_mes_luz['count'] / 2000).round(2)
ingresos_mes_luz['horas_m'] = ingresos_mes_luz.horas * 2000

In [38]:
ingresos_mes_luz.to_csv('../output/ingresos_mes_luz.csv',index=False)

In [39]:
fig_luz = go.Figure(data=[
    go.Bar(name="ingresos",x=ingresos_mes_luz['mes_n'],y=ingresos_mes_luz['count'],
           marker_color='skyblue'
          
          ),
    go.Line(name = 'horas luz',y=ingresos_mes_luz["horas_m"], x=ingresos_mes_luz["mes_n"],
            marker_color='yellow'
           )   
    
])
fig_luz.update_layout(barmode='group')

/home/sinatxe/anaconda3/envs/ironhack/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [40]:
fig_luz.write_html('../output/fig_luz.html')
    

In [41]:
fig = px.bar(ingresos_mes_luz, x="mes_n", y="count",color = 'count')
fig.show()

In [42]:
#morbi.groupby(['year','month']).agg({'day':'count'}).reset_index()